Librerías:

In [11]:
!pip install "dask[complete]"

import hashlib
import os
import time
import random
import string
import dask
from dask import delayed
import sys

Funciones auxiliares:

In [12]:
def generar_palabra_aleatoria(longitud_max=10):
    """Genera una palabra aleatoria de longitud variable (solo minúsculas ASCII)."""
    longitud = random.randint(3, longitud_max)
    return ''.join(random.choice(string.ascii_lowercase) for _ in range(longitud))

def crear_archivo_texto(nombre_archivo, num_palabras):
    """
    Crea un archivo de texto con un número específico de palabras aleatorias.
    Escribe directamente en BYTES con codificación UTF-8 para consistencia en el hashing.
    Los archivos se guardan en el directorio actual (/content/ en Colab).
    """
    print(f"Generando archivo '{nombre_archivo}' ({num_palabras:,} palabras)...")
    start_time = time.perf_counter()
    try:
        # Usar 'wb' para escribir bytes directamente.
        with open(nombre_archivo, 'wb') as f:
            palabras_generadas = 0
            buffer_size = 10000
            espacio = b' ' # Espacio como bytes
            while palabras_generadas < num_palabras:
                palabras_a_generar = min(buffer_size, num_palabras - palabras_generadas)
                # Generar palabras y convertirlas a bytes UTF-8
                buffer = [generar_palabra_aleatoria().encode('utf-8') for _ in range(palabras_a_generar)]
                f.write(espacio.join(buffer))
                # Añadir espacio entre bloques si no es el último del archivo
                if palabras_generadas + palabras_a_generar < num_palabras:
                    f.write(espacio)
                palabras_generadas += palabras_a_generar
                # Imprimir progreso para archivos grandes
                if num_palabras >= 100000 and palabras_generadas % (num_palabras // 10) == 0:
                     print(f"  ... {palabras_generadas / num_palabras * 100:.0f}% generado")

        end_time = time.perf_counter()
        print(f"Archivo '{nombre_archivo}' generado en {end_time - start_time:.4f} seg.")
        return True # Indicar éxito
    except Exception as e:
        print(f"\nError al generar el archivo '{nombre_archivo}': {e}")
        return False # Indicar fallo

def limpiar_archivos_temporales(prefijo="temp_colab_hash_dask_"):
    """Elimina los archivos de texto generados en /content/."""
    archivos_a_eliminar = [f for f in os.listdir('.') if f.startswith(prefijo) and f.endswith(".txt")]
    if not archivos_a_eliminar:
        print("\nNo se encontraron archivos temporales para eliminar.")
        return
    print("\nLimpiando archivos temporales...")
    for archivo in archivos_a_eliminar:
        try:
            os.remove(archivo)
            print(f" - Archivo '{archivo}' eliminado.")
        except OSError as e:
            print(f"Error al eliminar el archivo '{archivo}': {e}")
    print("Limpieza completada.")

def procesar_archivo_hash(nombre_archivo, num_palabras):
    """
    Lee un archivo (como bytes), calcula su hash SHA3-512 y mide los tiempos.
    Diseñada para ser ejecutada en paralelo por Dask.
    """
    # Mensaje de inicio (útil para ver paralelismo en Colab)
    # print(f"Dask iniciando: {os.path.basename(nombre_archivo)}")
    tiempos = {}
    datos = {'#palabras': num_palabras}
    hash_generado = ""
    mensaje_bytes = b""
    tiempo_total_inicio_archivo = time.perf_counter()

    # 1. Leer archivo (como bytes)
    tiempo_e1_inicio = time.perf_counter()
    try:
        # Usar 'rb' para leer bytes directamente
        with open(nombre_archivo, 'rb') as f:
            mensaje_bytes = f.read()
        datos['#caracteres_entrada'] = len(mensaje_bytes)
        tiempo_e1_fin = time.perf_counter()
        tiempos['T-E1'] = tiempo_e1_fin - tiempo_e1_inicio
    except Exception as e:
        print(f"\nError Dask [Leer] {nombre_archivo}: {e}")
        tiempos['T-E1'] = -1
        datos['#caracteres_entrada'] = -1

    # 2. Calcular Hash SHA3-512
    tiempo_e2_inicio = time.perf_counter()
    try:
        hash_obj = hashlib.sha3_512()
        hash_obj.update(mensaje_bytes)
        hash_generado = hash_obj.hexdigest()
        datos['#caracteres_salida'] = len(hash_generado) # Siempre 128
        tiempo_e2_fin = time.perf_counter()
        tiempos['T-E2'] = tiempo_e2_fin - tiempo_e2_inicio
    except Exception as e:
        print(f"\nError Dask [Hash] {nombre_archivo}: {e}")
        tiempos['T-E2'] = -1
        datos['#caracteres_salida'] = -1
        hash_generado = "Error"

    tiempo_total_fin_archivo = time.perf_counter()
    tiempos['T-Total'] = tiempo_total_fin_archivo - tiempo_total_inicio_archivo
    # Mensaje de fin (útil para ver paralelismo)
    # print(f"Dask finalizó: {os.path.basename(nombre_archivo)} en {tiempos['T-Total']:.4f}s")

    # Devolver un diccionario con todos los resultados para este archivo
    return {
        '#palabras': datos.get('#palabras', 'N/A'),
        '#caracteres_entrada': datos.get('#caracteres_entrada', 'N/A'),
        '#caracteres_salida': datos.get('#caracteres_salida', 128),
        'hash': hash_generado,
        'T-E1': tiempos.get('T-E1', -1),
        'T-E2': tiempos.get('T-E2', -1),
        'T-Total': tiempos.get('T-Total', -1),
        'nombre_archivo': nombre_archivo # Guardar nombre para referencia
    }

print("Funciones auxiliares definidas.")

Funciones auxiliares definidas.


Ejecución y resultados:

In [13]:
# Lista de número de palabras para probar
lista_num_palabras = [10, 100, 1000, 10000, 100000, 1000000, 10000000]
# lista_num_palabras = [10, 100, 1000] # Para pruebas rápidas en Colab

# Prefijo para los archivos generados
prefijo_archivo="temp_colab_hash_dask_"

# --- 1. Generación de Archivos de Prueba (Secuencial) ---
print("\n--- 1. Generación de Archivos de Prueba ---")
archivos_generados_info = []
tiempo_gen_inicio = time.perf_counter()
for num_palabras in lista_num_palabras:
    nombre_archivo = f"{prefijo_archivo}{num_palabras}.txt"
    if crear_archivo_texto(nombre_archivo, num_palabras):
        # Guardamos nombre y número de palabras para las tareas Dask
        archivos_generados_info.append({'nombre': nombre_archivo, 'palabras': num_palabras})
    else:
        print(f"\n¡FALLO CRÍTICO! No se pudo crear {nombre_archivo}. Abortando script.")
        # Limpiar lo que se haya podido crear
        limpiar_archivos_temporales(prefijo=prefijo_archivo)
        # Detener la ejecución de la celda (o el script si no es interactivo)
        raise SystemExit(f"Error al crear archivo {nombre_archivo}")

tiempo_gen_fin = time.perf_counter()
print(f"\n--- Tiempo Total Generación Archivos: {tiempo_gen_fin - tiempo_gen_inicio:.4f} segundos ---")

# --- 2. Mostrar Contenido para Verificación Manual (Archivo Pequeño) ---
print("\n--- 2. Contenido para Verificación Manual ---")
archivo_pequeno = f"{prefijo_archivo}{lista_num_palabras[0]}.txt"
contenido_para_copiar = ""
try:
    # Leer como TEXTO para mostrarlo fácilmente (usando utf-8, igual que al escribir)
    with open(archivo_pequeno, 'r', encoding='utf-8') as f:
        contenido_para_copiar = f.read()
    print(f"Copia el siguiente texto del archivo '{archivo_pequeno}' y pégalo en la herramienta online:")
    print("-" * 20 + " INICIO TEXTO " + "-" * 20)
    print(contenido_para_copiar)
    print("-" * 20 + " FIN TEXTO " + "-" * 20)
    print("\nHerramienta online: https://emn178.github.io/online-tools/sha3_512.html")
    print("\nHerramienta online: https://codebeautify.org/sha3-512-hash-generator")
except Exception as e:
    print(f"\nError al leer el archivo '{archivo_pequeno}' para mostrar contenido: {e}")
    print("No se pudo mostrar el contenido para la verificación manual.")

# --- 3. Crear Tareas Dask Delayed ---
print("\n--- 3. Creando Tareas Dask Delayed ---")
tareas_delayed = []
for archivo_info in archivos_generados_info:
    tarea = delayed(procesar_archivo_hash)(archivo_info['nombre'], archivo_info['palabras'])
    tareas_delayed.append(tarea)
print(f"Se crearon {len(tareas_delayed)} tareas Dask.")

# --- 4. Ejecutar Tareas en Paralelo con Dask ---
print("\n--- 4. Iniciando Cómputo Paralelo con Dask ---")
print("(Puede tomar tiempo, especialmente para archivos grandes...)")
tiempo_dask_inicio = time.perf_counter()
# Ejecutar todas las tareas creadas previamente
resultados_brutos = dask.compute(*tareas_delayed)
tiempo_dask_fin = time.perf_counter()
print(f"\n--- Cómputo Dask Completado ---")
print(f"Tiempo Total Cómputo Dask: {tiempo_dask_fin - tiempo_dask_inicio:.4f} segundos")

# --- 5. Procesar y Ordenar Resultados ---
print("\n--- 5. Procesando Resultados ---")
# Convertir la tupla de resultados a lista y filtrar posibles errores (aunque la func actual no retorna None)
resultados = [res for res in list(resultados_brutos) if isinstance(res, dict)]

# Ordenar los resultados por número de palabras para la tabla
resultados.sort(key=lambda x: x.get('#palabras', 0))
print("Resultados ordenados.")

# --- 6. Mostrar Hash Calculado para Verificación Manual ---
print("\n--- 6. Hash Calculado para Verificación Manual ---")
hash_calculado_pequeno = "No encontrado"
for res in resultados:
    if res.get('#palabras') == lista_num_palabras[0]:
        hash_calculado_pequeno = res.get('hash', 'Error en cálculo')
        break

if contenido_para_copiar: # Solo si pudimos leer el contenido antes
    print(f"Compara el hash de la herramienta online (para el texto mostrado arriba) con este hash calculado:")
    print(f"Hash calculado para '{archivo_pequeno}' ({lista_num_palabras[0]} palabras):")
    print(f"  {hash_calculado_pequeno}")
else:
    print("No se muestra hash para comparación manual porque no se pudo leer el contenido del archivo pequeño.")


# --- 7. Generar Tabla Resumen ---
print("\n" + "=" * 105)
print("                                   TABLA RESUMEN (Tiempos en segundos)")
print("=" * 105)
nombre_funcion_hash = "SHA3-512 (procesado con Dask en Colab)"
print(f"Función Hash Utilizada: {nombre_funcion_hash}")
print("Nota: '#Char Salida' es la longitud del hash hexadecimal (128). T-Total es por archivo.")
print("-" * 105)
print(f"{'#Palabras':>12} {'#Bytes Entr':>12} {'#Char Salida':>12} {'T-E1 (Leer)':>18} {'T-E2 (Hash)':>18} {'T-Total (Archivo)':>18}")
print("-" * 105)

# Imprimir filas de datos
for res in resultados:
    palabras_str = f"{res['#palabras']:,}" if isinstance(res['#palabras'], int) else str(res['#palabras'])
    char_in_str = f"{res['#caracteres_entrada']:,}" if isinstance(res['#caracteres_entrada'], int) else str(res['#caracteres_entrada'])
    char_out_str = str(res.get('#caracteres_salida','N/A')) # Usar get para seguridad
    t1_str = f"{res['T-E1']:.6f}" if isinstance(res['T-E1'], float) and res['T-E1'] >= 0 else "Error"
    t2_str = f"{res['T-E2']:.6f}" if isinstance(res['T-E2'], float) and res['T-E2'] >= 0 else "Error"
    tt_str = f"{res['T-Total']:.6f}" if isinstance(res['T-Total'], float) and res['T-Total'] >= 0 else "Error"

    print(f"{palabras_str:>12} {char_in_str:>12} {char_out_str:>12} {t1_str:>18} {t2_str:>18} {tt_str:>18}")

print("=" * 105)
print(f"Tiempo total del cómputo paralelo Dask (excluyendo generación archivos): {tiempo_dask_fin - tiempo_dask_inicio:.4f} segundos")

# --- Comparación Manual de Hashes ---
print("\n--- Comparación Manual de Hashes ---")
print("Ingrese el hash generado por la herramienta en línea para comparar con los resultados del programa.")

while True:
    # Solicitar al usuario el número de palabras del archivo que desea verificar
    try:
        num_palabras = int(input("Ingrese el número de palabras del archivo que desea verificar (o 0 para salir): "))
        if num_palabras == 0:
            print("Saliendo de la comparación manual.")
            break
    except ValueError:
        print("Por favor, ingrese un número válido.")
        continue

    # Buscar el resultado correspondiente al número de palabras
    resultado = next((res for res in resultados if res['#palabras'] == num_palabras), None)
    if not resultado:
        print(f"No se encontró un archivo con {num_palabras} palabras. Intente nuevamente.")
        continue

    # Mostrar el hash calculado por el programa
    print(f"Hash calculado por el programa para {num_palabras} palabras: {resultado['hash']}")

    # Solicitar al usuario el hash generado por la herramienta en línea
    hash_usuario = input("Ingrese el hash generado por la herramienta en línea: ").strip()

    # Comparar los hashes
    if hash_usuario == resultado['hash']:
        print("¡Los hashes coinciden! El archivo fue procesado correctamente.")
    else:
        print("Los hashes NO coinciden. Verifique el contenido del archivo y el hash ingresado.")

# --- 8. Limpiar Archivos Temporales ---
limpiar_archivos_temporales(prefijo=prefijo_archivo)


--- 1. Generación de Archivos de Prueba ---
Generando archivo 'temp_colab_hash_dask_10.txt' (10 palabras)...
Archivo 'temp_colab_hash_dask_10.txt' generado en 0.0009 seg.
Generando archivo 'temp_colab_hash_dask_100.txt' (100 palabras)...
Archivo 'temp_colab_hash_dask_100.txt' generado en 0.0007 seg.
Generando archivo 'temp_colab_hash_dask_1000.txt' (1,000 palabras)...
Archivo 'temp_colab_hash_dask_1000.txt' generado en 0.0051 seg.
Generando archivo 'temp_colab_hash_dask_10000.txt' (10,000 palabras)...
Archivo 'temp_colab_hash_dask_10000.txt' generado en 0.0350 seg.
Generando archivo 'temp_colab_hash_dask_100000.txt' (100,000 palabras)...
  ... 10% generado
  ... 20% generado
  ... 30% generado
  ... 40% generado
  ... 50% generado
  ... 60% generado
  ... 70% generado
  ... 80% generado
  ... 90% generado
  ... 100% generado
Archivo 'temp_colab_hash_dask_100000.txt' generado en 0.3722 seg.
Generando archivo 'temp_colab_hash_dask_1000000.txt' (1,000,000 palabras)...
  ... 10% generado
